In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
import numpy as np

In [ ]:
timesData = pd.read_csv("dataframe/timesData.csv")
timesData.head()

In [ ]:
timesData.info()

In [ ]:
timesData['income']=pd.to_numeric(timesData['income'], errors='coerce')

In [ ]:
shanghaiData = pd.read_csv("dataframe/shanghaiData.csv")
shanghaiData.head()

In [ ]:
school_and_country = pd.read_csv("dataframe/school_and_country_table.csv")
school_and_country.head()

In [ ]:
cwurData = pd.read_csv("dataframe/cwurData.csv")
cwurData.head()

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
education_expenditure = pd.read_csv("dataframe/education_expenditure_supplementary_data.csv",engine='python')
education_expenditure.head()

1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframe

In [147]:
most_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmax())]
most_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,11
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,NaN
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,NaN
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,NaN
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,NaN


In [ ]:
least_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmin())]
least_recent_data.head()

In [ ]:
##timesData[timesData['year']>=timesData['year'].max()]

In [ ]:
##timesData[timesData['university_name']=='Aalto University']

In [ ]:
#timesData.groupby('university_name').min()['year']


In [ ]:
#timesData.groupby('university_name').max()['year']

2. For each university, compute the improvement in "income" between the least recent and the most recent data points

In [ ]:
punto2=pd.merge(most_recent_data, least_recent_data, on='university_name')
punto2.head()

In [ ]:
punto2.columns

In [ ]:
##nuovo_punto2=punto2[punto2[['income_x','income_y']].notnull()]
##nuovo_punto2.head()

In [ ]:
##def calcola_perc(riga):
    ##return riga['income_x'] - riga['income_y']

In [ ]:
##nuovo_punto2['income_improvement']=nuovo_punto2.apply(calcola_perc, axis=1)
##nuovo_punto2.head()

In [ ]:
##nuovo_punto2['differenza']=nuovo_punto2['income_x']-nuovo_punto2['income_y']
##nuovo_punto2

In [ ]:
#nuovo_punto2[nuovo_punto2['income_y']-nuovo_punto2['income_x']]

In [ ]:
##lista=nuovo_punto2['income_x']
##lista


In [ ]:
##nuovo=lista.astype(float)
##nuovo

In [ ]:
def calcola_perc(riga):
    return riga['votes_first_vote']/ risultato[riga['state']] *100

In [ ]:
##nuovo_punto2['income_y']=nuovo_punto2['income_y'].to_numeric()

In [ ]:
##nuovo_punto2=np.array(list, dtype=float)

In [ ]:
punto2['income_improvement']=punto2['income_y']-punto2['income_x']
punto2.head()

3. Find the university with the largest increase computed in the previous point

In [ ]:
punto2.loc[punto2['income_improvement']>= punto2['income_improvement'].max()]['university_name'] #possiamo anche togliere l'ultima quadra


4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [ ]:
most_recent_s=shanghaiData.loc[(shanghaiData.groupby('university_name')['year'].idxmax())]
most_recent_c=cwurData.loc[(cwurData.groupby('institution')['year'].idxmax())]
#most_recent_data

In [ ]:
##most_recent_data[most_recent_data['university_name'] == 'Aarhus University']
##most_recent_s[most_recent_s['university_name'] == 'Aarhus University']
##most_recent_c[most_recent_c['institution'] == 'Aarhus University']

In [ ]:
most_recent_data['world_rank']

In [148]:
lista = most_recent_data['world_rank']
df =pd.DataFrame(lista)
df.head()

,world_rank
2405,601-800
2003,201-250
2056,251-300
1908,=106
2105,301-350


In [149]:
df['min_range'] = df['world_rank'].str.extract(r'^\=?(\d+)')
df['max_range'] = df['world_rank'].str.extract(r'(\d+$)')
df.head()

,world_rank,min_range,max_range
2405,601-800,601,800
2003,201-250,201,250
2056,251-300,251,300
1908,=106,106,106
2105,301-350,301,350


In [150]:
df['min_range']=pd.to_numeric(df['min_range'], errors='coerce')
df['max_range']=pd.to_numeric(df['max_range'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 2405 to 2134
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   world_rank  818 non-null    object
 1   min_range   818 non-null    int64 
 2   max_range   818 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 25.6+ KB


In [151]:
most_recent_data['world_rank_t'] = (df['max_range'] + df['min_range'])//2
most_recent_data['world_rank_t'].head()

2405    700
2003    225
2056    275
1908    106
2105    325
Name: world_rank_t, dtype: int64

In [156]:
#rifacciamo per s
lista_s = most_recent_s['world_rank']
df_s =pd.DataFrame(lista_s)
df_s['min_range'] = df_s['world_rank'].str.extract(r'^\=?(\d+)')
df_s['max_range'] = df_s['world_rank'].str.extract(r'(\d+$)')
df_s['min_range']=pd.to_numeric(df_s['min_range'], errors='coerce')
df_s['max_range']=pd.to_numeric(df_s['max_range'], errors='coerce')
most_recent_s['world_rank_t'] = (df_s['max_range'] + df_s['min_range'])//2
most_recent_s.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_t
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015,350
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015,450
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,73
4497,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015,125
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011,126


In [164]:
most_recent_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024 entries, 1981 to 1838
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            1024 non-null   int64  
 1   institution           1024 non-null   object 
 2   country               1024 non-null   object 
 3   national_rank         1024 non-null   int64  
 4   quality_of_education  1024 non-null   int64  
 5   alumni_employment     1024 non-null   int64  
 6   quality_of_faculty    1024 non-null   int64  
 7   publications          1024 non-null   int64  
 8   influence             1024 non-null   int64  
 9   citations             1024 non-null   int64  
 10  broad_impact          1023 non-null   float64
 11  patents               1024 non-null   int64  
 12  score                 1024 non-null   float64
 13  year                  1024 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 120.0+ KB


5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [ ]:
#most_recent_data.head()

In [ ]:
#most_recent_data['year']

In [ ]:
#rapporto=most_recent_data['female_male_ratio']
#rapporto.head()

In [ ]:
#dataframe=pd.DataFrame(rapporto)
#dataframe.head()

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['female'] = most_recent_data['female_male_ratio'].str.extract(r'(^\d+)')
most_recent_data['male'] = most_recent_data['female_male_ratio'].str.extract(r'(\d+$)')
most_recent_data.head()

In [ ]:
most_recent_data['male']=pd.to_numeric(most_recent_data['male'], errors='coerce')
most_recent_data['female']=pd.to_numeric(most_recent_data['female'], errors='coerce')

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['num_students'] = most_recent_data['num_students'].str.replace(',','')
most_recent_data.head()

In [ ]:
most_recent_data['num_students']=pd.to_numeric(most_recent_data['num_students'], errors='coerce')
most_recent_data.head()

In [ ]:
##def calc_numero(row):
    ##return most_recent_data['num_students']* most_recent_data[row]//100

In [ ]:
#most_recent_data['num_male']=most_recent_data['male'].apply(calc_numero, axis= 1)

In [ ]:
most_recent_data['num_male']= most_recent_data['male'] * most_recent_data['num_students'] // 100
most_recent_data.head()

In [ ]:
most_recent_data['num_fem']= most_recent_data['female'] * most_recent_data['num_students'] // 100
most_recent_data.head()

6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [ ]:
most_recent_data['rapporto'] = most_recent_data['num_fem'] / most_recent_data['num_male']
media = most_recent_data['rapporto'].notnull().mean()

In [ ]:
punto6 = most_recent_data.loc[most_recent_data['rapporto'] <= media]
punto6.head()

In [ ]:
punto6['university_name']

7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [ ]:
#denominatore
deno = most_recent_data.groupby('country', as_index = True)['num_students'].sum()
#numeratore
deno

In [ ]:
num=punto6.groupby('country')['num_students'].sum()

In [ ]:
num.dtype

In [ ]:
deno.dtype

In [ ]:
fraction=num/deno
fraction

In [ ]:
#most_recent_data.loc[most_recent_data.groupby('country').sum()['num_students']]

8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
#educational_attainment.dropna(educational_attainment,how="any", thresh=None, subset=(['country_name','series_name']), inplace=True)
#educational_attainment.head() 

In [ ]:
educ = educational_attainment[(educational_attainment['country_name'].notnull()) | (educational_attainment['series_name'].notnull())]
educ.head()

In [ ]:
#educational_attainment.dropna?


9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [ ]:
ed_pulito= educ.dropna(axis='columns', how='all')
ed_pulito.head()

In [ ]:
#ed_pulito[ed_pulito['1986'].notnull()]

In [ ]:
pd.melt?

In [ ]:
ed_pulito.columns[2:]

In [ ]:
pd.melt(ed_pulito, id_vars=['country_name','series_name'], value_vars=['1985', '1986', '1987', '1990', '1991', '1992', '1993', '1995', '1996',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013'])


In [ ]:
ed_pulito.transpose?

10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [ ]:
#tab=pd.merge(timesData, shanghaiData, on='university_name')
#tab_merge=pd.merge(tab, cwurData, left_on='university_name', right_on='institution')
#tab_merge

In [ ]:
tabella_merge=pd.merge(school_and_country, timesData, left_on='school_name', right_on='university_name', how='left')
tabella_merge.loc[tabella_merge['university_name'].isnull()]

In [ ]:
tabella_merge=pd.merge(school_and_country, shanghaiData, left_on='school_name', right_on='university_name', how='left')
tabella_merge.head()

In [ ]:
tabella_merge.loc[tabella_merge['university_name'].isnull()]

In [ ]:
tabella_merge3=pd.merge(tabella_merge, cwurData, left_on='school_name', right_on='institution', how='left')
tabella_merge3.head()

In [ ]:
tabella_merge3.columns

In [ ]:
punto10=tabella_merge3.loc[(tabella_merge3['institution'].isnull())|(tabella_merge3['university_name'].isnull())][['university_name','institution','school_name']]


In [ ]:
punto10.loc[(punto10.groupby('school_name')['institution'].unique())]

In [ ]:
#pd.punto10(['school_name'], index=False).unique()
pd.unique(punto10['school_name'])
#punto10.['school_name'].unique

11. In the times ranking, compute the number of times each university appears

In [ ]:
punto11 = timesData.groupby('university_name', as_index = False)['world_rank'].count()
punto11

12. Find the universities that appear at most twice in the times ranking.

In [ ]:
punto12 = punto11[punto11['world_rank']<= 2]
punto12

13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

4-10-13